In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
train = pd.read_csv('/content/drive/My Drive/train.csv')
test = pd.read_csv('/content/drive/My Drive/test.csv')
sample_submission = pd.read_csv('/content/drive/My Drive/sample_submission.csv')
print(train.shape)
print(test.shape)
print(sample_submission.shape)

(159571, 8)
(153164, 2)
(153164, 7)


In [3]:

non_toxic = []
indexes = train.index
attributes = train.columns
toxicity = ['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']
# print(attributes)
for i in indexes:
  flag = 0
  for j in attributes:
    if j in toxicity:
      if(train.loc[i][j]==1):
        non_toxic.append(0)
        flag = 1
        break
  if(flag == 0):
    non_toxic.append(1)
print(len(non_toxic))


159571


In [4]:
#
print(len(train.columns))
train.insert(8, "non_toxic", non_toxic, True)
print(len(train.columns))

8
9


In [5]:
print("Any missing values in training set:",train.isnull().values.any())
print("Any missing values in test set:",test.isnull().values.any(), "\n")

Any missing values in training set: False
Any missing values in test set: False 



In [24]:
import re
import string
temp = ([{string.punctuation}])
print(temp)
re_new = re.compile(f'([{string.punctuation}])')
def tokenize(s): return re_new.sub(' \1 ', s).split()
# def tokenize(s): return re_new.sub(s).split()

[{'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'}]


In [0]:
from nltk.tokenize import RegexpTokenizer
n = train.shape[0]
# print(n)

vec = TfidfVectorizer(ngram_range=(1,2), tokenizer= tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
train_term_doc = vec.fit_transform(train['comment_text'])
test_term_doc = vec.transform(test['comment_text'])

In [26]:
print(train_term_doc)
print(test_term_doc)

  (0, 605)	0.09554896532881822
  (0, 24812)	0.1106120878711465
  (0, 754)	0.10854842526372448
  (0, 23629)	0.12176298365103679
  (0, 495)	0.12025660428336654
  (0, 26386)	0.11569800392263023
  (0, 1120)	0.11467471067118694
  (0, 227247)	0.06759576617142528
  (0, 277168)	0.16840057590137794
  (0, 201372)	0.17114473997599935
  (0, 11201)	0.051393125279668044
  (0, 166567)	0.04236364181253797
  (0, 294758)	0.09316943388910333
  (0, 247592)	0.13875806755423012
  (0, 313961)	0.056633812011729436
  (0, 329299)	0.07665655627718654
  (0, 144101)	0.060726562694642244
  (0, 315470)	0.13618994328915118
  (0, 329374)	0.09630117531627859
  (0, 274210)	0.08246823528371676
  (0, 312626)	0.1200532061591735
  (0, 118042)	0.04782563569642296
  (0, 255178)	0.09431222436044351
  (0, 50627)	0.1044554877864965
  (0, 1883)	0.03610765384594172
  :	:
  (159570, 19616)	0.0797181152108849
  (159570, 331579)	0.08955222378050001
  (159570, 20158)	0.055335961190675305
  (159570, 331578)	0.07154858046882678
  (15957

In [27]:
def prob(y_i, y):
  # print(y_i,y)
  p = x[y==y_i].sum(0)
  print(type(y==y_i))
  print(y==y_i)
  print(p)
  return (p+1) / ((y==y_i).sum()+1)
x = train_term_doc
test_x = test_term_doc
# pr(1,y)
def get_value(y):
  y = y.values
  r = np.log(pr(1,y) / prob(0,y))
  m = LogisticRegression(C=4, dual=True)
  x_nb = x.multiply(r)
  return m.fit(x_nb, y), r
preds = np.zeros((len(test), len(toxicity)))

for i, j in enumerate(toxicity):
    print('fit', j)
    m,r = get_value(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit toxic
<class 'numpy.ndarray'>
[False False False ... False False False]
[[413.16166202   5.07989104   2.26768616 ...   0.           0.
    0.        ]]
<class 'numpy.ndarray'>
[ True  True  True ...  True  True  True]
[[4.26156350e+03 7.93308576e+01 5.22848041e+01 ... 3.99301467e-01
  3.19218811e+00 3.19218811e+00]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


fit severe_toxic
<class 'numpy.ndarray'>
[False False False ... False False False]
[[44.49954305  0.95040819  0.18992393 ...  0.          0.
   0.        ]]
<class 'numpy.ndarray'>
[ True  True  True ...  True  True  True]
[[4.63022561e+03 8.34603405e+01 5.43625663e+01 ... 3.99301467e-01
  3.19218811e+00 3.19218811e+00]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


fit obscene
<class 'numpy.ndarray'>
[False False False ... False False False]
[[228.18820287   2.9786698    1.16422224 ...   0.           0.
    0.        ]]
<class 'numpy.ndarray'>
[ True  True  True ...  True  True  True]
[[4.44653695e+03 8.14320789e+01 5.33882680e+01 ... 3.99301467e-01
  3.19218811e+00 3.19218811e+00]]


...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


fit insult
<class 'numpy.ndarray'>
[False False False ... False False False]
[[196.46140779   2.5646008    0.89865972 ...   0.           0.
    0.        ]]
<class 'numpy.ndarray'>
[ True  True  True ...  True  True  True]
[[4.47826375e+03 8.18461479e+01 5.36538305e+01 ... 3.99301467e-01
  3.19218811e+00 3.19218811e+00]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


fit identity_hate
<class 'numpy.ndarray'>
[False False False ... False False False]
[[34.56445382  0.60038758  0.06294123 ...  0.          0.
   0.        ]]
<class 'numpy.ndarray'>
[ True  True  True ...  True  True  True]
[[4.64016070e+03 8.38103611e+01 5.44895490e+01 ... 3.99301467e-01
  3.19218811e+00 3.19218811e+00]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [30]:
submission_id = pd.DataFrame({'id': sample_submission["id"]})
# print(submid.head(10))
submission_toxic = pd.DataFrame(preds, columns = toxicity)
submission = pd.concat([submission_id, submission_toxic],axis = 1)
print(submission.head(10))
submission.to_csv('/content/drive/My Drive/submission.csv', index=False)

    


                 id     toxic  severe_toxic  ...    threat    insult  identity_hate
0  00001cee341fdb12  0.999993      0.045162  ...  0.002365  0.972025       0.116406
1  0000247867823ef7  0.001132      0.000259  ...  0.000088  0.001281       0.000274
2  00013b17ad220c46  0.003289      0.000345  ...  0.000113  0.001560       0.000404
3  00017563c3f7919a  0.001047      0.000222  ...  0.000177  0.001023       0.000271
4  00017695ad8997eb  0.010655      0.000504  ...  0.000130  0.002361       0.000337
5  0001ea8717f6de06  0.005314      0.000266  ...  0.000371  0.002921       0.000337
6  00024115d4cbde0f  0.000394      0.000166  ...  0.000091  0.000702       0.000348
7  000247e83dcc1211  0.517880      0.000442  ...  0.000143  0.012646       0.000338
8  00025358d4737918  0.015155      0.000216  ...  0.000161  0.003916       0.000571
9  00026d1092fe71cc  0.002820      0.000227  ...  0.000111  0.002646       0.000262

[10 rows x 7 columns]
